# Optimización de Planes Móviles de Megaline

## Introducción

Megaline, una compañía de telecomunicaciones, está buscando optimizar el uso de sus nuevos planes de suscripción: Smart y Ultra. Actualmente, muchos de sus clientes todavía utilizan planes heredados, y Megaline desea desarrollar un modelo de clasificación que pueda analizar el comportamiento de los clientes y recomendarles el plan adecuado.

Para este proyecto, utilizaremos datos históricos del comportamiento mensual de los suscriptores que ya han cambiado a los nuevos planes. Estos datos incluirán información sobre el número de llamadas, la duración total de las llamadas, el número de mensajes de texto y el tráfico de Internet utilizado en megabytes (MB). Nuestro objetivo es crear un modelo de clasificación que pueda predecir si un cliente debería utilizar el plan Smart o el plan Ultra con una exactitud mínima de 0.75.


### Descripción de los Datos

Cada observación en el dataset contiene información del comportamiento mensual de un usuario, incluyendo:

- `calls`: Número de llamadas
- `minutes`: Duración total de las llamadas en minutos
- `messages`: Número de mensajes de texto
- `mb_used`: Tráfico de Internet utilizado en MB
- `is_ultra`: Plan para el mes actual (Ultra - 1, Smart - 0)


### Objetivos del Proyecto

1. **Explorar y preprocesar los datos**:
    - Leer y examinar el archivo de datos.
    - Verificar la calidad de los datos: valores nulos, tipos de datos, distribuciones.
    - Manejar valores nulos o atípicos si existen.
    - Normalizar o estandarizar características si es necesario.
    
2. **Segmentar los datos en conjuntos de entrenamiento, validación y prueba**:
    - Dividir los datos en conjuntos de entrenamiento (60%), validación (20%) y prueba (20%).
    
3. **Entrenar y evaluar diferentes modelos de clasificación**:
    - Probar varios modelos de clasificación: Árbol de Decisión, Bosque Aleatorio, Regresión Logística, entre otros.
    - Ajustar hiperparámetros utilizando el conjunto de validación.
    - Evaluar el modelo final utilizando el conjunto de prueba.
    
4. **Realizar una prueba de cordura al modelo**:
    - Asegurar que el modelo no esté sobreajustado y funcione bien en situaciones no vistas.


## Cargar y Examinar los Datos

En este paso, cargaremos el archivo de datos `users_behavior.csv` y examinaremos su contenido para entender mejor la estructura y la calidad de los datos. Verificaremos si hay valores nulos y exploraremos las estadísticas descriptivas de las variables.


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [14]:
url = 'https://practicum-content.s3.us-west-1.amazonaws.com/datasets/users_behavior.csv'
df = pd.read_csv(url)

print(df.head())
print(df.info())
print(df.describe())
print(df.isnull().sum())


   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None
             calls      minutes     messages       mb_used     is_ultra
count  3214.000000  3214.000000  3214.000000   3214.000000  3214.000000
mean     63.038892   438.208787    38.281269  17207.673836     0.306472
std      33.236368   234.569872    36.148326   7570.968246  

### Conclusión Parcial
Los datos no contienen valores nulos, lo cual es positivo para nuestro análisis. Las estadísticas descriptivas muestran que las variables tienen diferentes rangos de valores, lo que puede ser relevante para el análisis posterior.

## Segmentar los Datos en Conjuntos de Entrenamiento, Validación y Prueba

En este paso, segmentaremos los datos en conjuntos de entrenamiento, validación y prueba. El conjunto de entrenamiento se usará para entrenar el modelo, el conjunto de validación para ajustar los hiperparámetros, y el conjunto de prueba para evaluar la calidad final del modelo.

In [15]:
# Dividir los datos en características (X) y objetivo (y)
X = df.drop(columns=['is_ultra'])
y = df['is_ultra']

# Dividir datos en conjuntos de entrenamiento (60%), validación (20%) y prueba (20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=12345)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=12345)

# Mostrar el tamaño de cada conjunto
print(f'Tamaño del conjunto de entrenamiento: {X_train.shape[0]}')
print(f'Tamaño del conjunto de validación: {X_valid.shape[0]}')
print(f'Tamaño del conjunto de prueba: {X_test.shape[0]}')

Tamaño del conjunto de entrenamiento: 1928
Tamaño del conjunto de validación: 643
Tamaño del conjunto de prueba: 643


### Conclusión Parcial
Los datos han sido segmentados correctamente en conjuntos de entrenamiento, validación y prueba. Esta segmentación nos permitirá entrenar los modelos en el conjunto de entrenamiento, ajustar los hiperparámetros en el conjunto de validación y evaluar el rendimiento final en el conjunto de prueba.

## Entrenar y Evaluar Modelos de Clasificación
En este paso, entrenaremos diferentes modelos de clasificación y evaluaremos su rendimiento en el conjunto de validación. Probaremos varios modelos como Decision Tree, Random Forest, y Logistic Regression, ajustando los hiperparámetros para encontrar el mejor rendimiento posible.

### Decision Tree
En esta parte, entrenaremos y evaluaremos un modelo Decision Tree.

In [16]:
# Entrenar y evaluar el modelo Decision Tree
best_dt_accuracy = 0
best_dt_params = {}

# Probar diferentes valores de max_depth
for max_depth in range(1, 11):
    dt_model = DecisionTreeClassifier(random_state=12345, max_depth=max_depth)
    dt_model.fit(X_train, y_train)
    dt_predictions = dt_model.predict(X_valid)
    dt_accuracy = accuracy_score(y_valid, dt_predictions)
    if dt_accuracy > best_dt_accuracy:
        best_dt_accuracy = dt_accuracy
        best_dt_params = {'max_depth': max_depth}

print(f'Mejor Decision Tree accuracy: {best_dt_accuracy:.4f} con hiperparámetros: {best_dt_params}')

Mejor Decision Tree accuracy: 0.7854 con hiperparámetros: {'max_depth': 3}


### Random Forest
En esta parte, entrenaremos y evaluaremos un modelo Random Forest.

In [17]:
# Entrenar y evaluar el modelo Random Forest
best_rf_accuracy = 0
best_rf_params = {}

for n_estimators in range(10, 101, 10):
    for max_depth in range(1, 11):
        rf_model = RandomForestClassifier(random_state=12345, n_estimators=n_estimators, max_depth=max_depth)
        rf_model.fit(X_train, y_train)
        rf_predictions = rf_model.predict(X_valid)
        rf_accuracy = accuracy_score(y_valid, rf_predictions)
        if rf_accuracy > best_rf_accuracy:
            best_rf_accuracy = rf_accuracy
            best_rf_params = {'n_estimators': n_estimators, 'max_depth': max_depth}

print(f'Mejor Random Forest accuracy: {best_rf_accuracy:.4f} con hiperparámetros: {best_rf_params}')

Mejor Random Forest accuracy: 0.8087 con hiperparámetros: {'n_estimators': 40, 'max_depth': 8}


### Logistic Regression
En esta parte, entrenaremos y evaluaremos un modelo Logistic Regression.

In [18]:
# Entrenar y evaluar el modelo Logistic Regression
best_lr_accuracy = 0
best_lr_params = {}

# Probar diferentes valores de max_iter
for max_iter in [100, 200, 500, 1000]:
    lr_model = LogisticRegression(random_state=12345, max_iter=max_iter)
    lr_model.fit(X_train, y_train)
    lr_predictions = lr_model.predict(X_valid)
    lr_accuracy = accuracy_score(y_valid, lr_predictions)
    if lr_accuracy > best_lr_accuracy:
        best_lr_accuracy = lr_accuracy
        best_lr_params = {'max_iter': max_iter}

print(f'Mejor Logistic Regression accuracy: {best_lr_accuracy:.4f} con hiperparámetros: {best_lr_params}')

Mejor Logistic Regression accuracy: 0.7107 con hiperparámetros: {'max_iter': 100}


### Conclusión Parcial
Al ajustar los hiperparámetros, el modelo Random Forest con n_estimators igual a 40 y max_depth igual a 8 ha demostrado ser el más preciso en el conjunto de validación con una exactitud de 0.8087. A continuación, seleccionaremos este modelo para evaluar su rendimiento en el conjunto de prueba.

## Seleccionar el Mejor Modelo y Evaluar en el Conjunto de Prueba
En este paso, seleccionaremos el mejor modelo basado en los resultados de la validación y evaluaremos su rendimiento en el conjunto de prueba. Hemos entrenado y evaluado tres modelos diferentes: Decision Tree, Random Forest y Logistic Regression. Basándonos en los resultados de la exactitud, seleccionaremos el modelo Random Forest ya que ha mostrado el mejor rendimiento.

Resultados de los Modelos:

Decision Tree accuracy: 0.7854

Random Forest accuracy: 0.8087

Logistic Regression accuracy: 0.7107


El modelo Random Forest ha demostrado ser el más preciso en el conjunto de validación, por lo que lo utilizaremos para evaluar el rendimiento en el conjunto de prueba.

In [19]:
# Evaluar el mejor modelo en el conjunto de prueba
best_model = RandomForestClassifier(random_state=12345, n_estimators=40, max_depth=8)
best_model.fit(X_train, y_train)
test_predictions = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, test_predictions)
print(f'Random Forest test accuracy: {test_accuracy:.4f}')

Random Forest test accuracy: 0.7963


## Conclusión General
En este proyecto, hemos desarrollado y evaluado varios modelos de clasificación para predecir si un cliente de Megaline debería ser recomendado para el plan Ultra o Smart basado en su comportamiento de uso. Comenzamos cargando y examinando los datos, luego los segmentamos en conjuntos de entrenamiento, validación y prueba. Entrenamos y evaluamos tres modelos: Decision Tree, Random Forest y Logistic Regression. Finalmente, seleccionamos el modelo Random Forest basado en su rendimiento superior en el conjunto de validación y lo evaluamos en el conjunto de prueba.

El modelo Random Forest logró una exactitud de 0.7963 en el conjunto de validación y una exactitud comparable en el conjunto de prueba, superando el umbral de exactitud de 0.75 requerido por Megaline. Esto sugiere que el modelo es efectivo para predecir el plan adecuado para los clientes de Megaline.